In [1]:
import pandas as pd
import numpy as np
import itertools
import os
from IPython.display import display, HTML


from Event_detection_functions import *


In [2]:
# Decleration of dataframes 
Event_df = pd.DataFrame(columns = ['timestamp','Phase','minuten_index',
                                   'Ladevorgang', 'Status', 'Ladeleistung',
                                   'minuten_index_Abschaltung','timestamp_abschalt'])
pseudocode_df = pd.DataFrame([])


In [3]:
#test loop, delete later
for minute_day in range (1,500):

    # Insert this in the update routine after the power_today_minute.csv export
    # The following variables have to be set to the corresponding dataframe in the environment (export to power_today_minute_x.csv)
    P_Phase_1 = pd.read_csv('power_today_minute_1.csv', names = ['timestamp','P'])
    P_Phase_1 = P_Phase_1[:minute_day]
    P_Phase_2 = pd.read_csv('power_today_minute_2.csv', names = ['timestamp','P'])
    P_Phase_2 = P_Phase_2[:minute_day]
    P_Phase_3 = pd.read_csv('power_today_minute_3.csv', names = ['timestamp','P'])
    P_Phase_3 = P_Phase_3[:minute_day]

    dict = {1:P_Phase_1,2:P_Phase_2,3:P_Phase_3}

    #Detection of switch-on and switch-off events
    for Phase in range (1,4):
        
        pseudocode_df = dict[Phase]
        pseudocode_df['P_delta'] = dict[Phase].P.diff()
        Event_df = detect_switch_event(pseudocode_df, Phase, Event_df)

        


/Users/markus/version-control/transients_classifier/Implementation_Events Kopie/Event_detection_functions.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  only_load_event_df['deviation_P'] = only_load_event_df['Ladeleistung'].apply(lambda ladeleistung_auto: abs(ladeleistung_auto-P_residual)/ladeleistung_auto)


In [4]:
Event_df

,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.507265e+09,2,407,ein,checked,3604,0,0
1,1.507267e+09,2,430,ein,checked,3476,0,0


In [5]:
ladevorgang_df = combine_charging_events(Event_df)
ladevorgang_df

/Users/markus/version-control/transients_classifier/Implementation_Events Kopie/Event_detection_functions.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  same_events['deviation_average'] = same_events['Ladeleistung'].                                     apply(lambda ladeleistung_auto: ladeleistung_auto/(same_events['Ladeleistung'].mean()))


,timestamp_start,Phasen,Durchschnittliche Ladeleistung,Ladevorgang_ende
0,1.507265e+09,[2],[2: 3604.0],0
1,1.507267e+09,[2],[2: 3476.0],0


In [7]:
#timestamp has to be replaced by variable used in py script
timestamp = (pd.read_csv('power_today_minute_1.csv', names = ['timestamp','P'])).iloc[500]['timestamp']

charging_df=cars_charging(timestamp,ladevorgang_df)
charging_df

,timestamp,cars_total,one_phase,two_phases,three_phases,cars_phase_1,cars_phase_2,cars_phase_3
0,1.507271e+09,2.0,2.0,0.0,0.0,0.0,2.0,0.0


In [8]:
#export
charging_df.to_csv('charging_df.csv', )
